In [ ]:
# 드라이브 마운팅
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# library loading

In [ ]:
 #우분투 업데이트-> 크롬 드라이버 설치 snap
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.IbQAUnKZSv/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.158MzmDQy2/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.HvWgvrLwhd/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian bust

In [ ]:
# 필수 라이브러리
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import csv
import time
import re # 정규표현식
import pandas as pd #csv저장
from datetime import datetime # 시간 체크
from pytz import timezone

# page로딩과 구조 로딩에 gab이 있어서 원하는 tag나올떄까지 기다리는거
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# driver setting

In [ ]:
# 화면 주소 정의
## 스킨케어 토너
a = 'https://www.oliveyoung.co.kr/store/display/getCategoryShop.do?dispCatNo=10000010001'
## 스킨케어 에센스
ab = 'https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100010009&isLoginCnt=0&aShowCnt=0&bShowCnt=0&cShowCnt=0'
## 스킨케어 미스트
ac = 'https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100010010&isLoginCnt=0&aShowCnt=0&bShowCnt=0&cShowCnt=0'

## 립메이크업
ba = 'https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100020006&isLoginCnt=0&aShowCnt=0&bShowCnt=0&cShowCnt=0'
## 베이스메이크업
bb='https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100020001&isLoginCnt=0&aShowCnt=0&bShowCnt=0&cShowCnt=0'
## 아이메이크업
bc='https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100020007&isLoginCnt=0&aShowCnt=0&bShowCnt=0&cShowCnt=0'

In [ ]:
#  driver setting
URL = 'https://www.oliveyoung.co.kr/store/main/main.do?oy=0' # 첫화면
ops = webdriver.ChromeOptions() 
ops.add_experimental_option('excludeSwitches', ['enable-logging'])
ops.add_argument('--headless')        # Head-less 설정
ops.add_argument('--no-sandbox')
ops.add_argument('--disable-dev-shm-usage')
# ops.add_argument('__headlees') ## 확인해보자

# chromedriver path
driver = webdriver.Chrome('chromedriver', options=ops)

# defined function

In [ ]:
# review count
def rv_num2():
  global q # 총 리뷰수
  global pn # 리뷰 페이지수
  global pc # 마지막 페이지의 리뷰수
  q = driver.find_element(By.XPATH, '//*[@id="reviewInfo"]/a/span').text[1:-1]
  q = int(re.sub(r"[^0-9]", "", q))
  if q == 0:  # 리뷰수가 없는 경우
      pn = 'zero'
      pc = 'zero'
  elif 0 < q <= 10:  # 리뷰수가 10이하인 경우
      pn = 'one'
      pc = q
  else: # 리뷰수가 11이상
    # page_num(리뷰 페이지 수), product_count(마지막페이지의 리뷰수)
      pn = (q // 10) + 1 
      pc = q % 10

In [ ]:
# reviwe information

def rv_info2():
  try :
      global pname   
      pname = driver.find_element(By.XPATH,'//*[@id="Contents"]/div[2]/div[2]/div/p[2]').text
      global tstar
      # #gdasContentsArea > div > div.product_rating_area.review-write-delete > div > div.star_area > p.num > strong
      tstar = driver.find_element(By.CSS_SELECTOR,'#gdasContentsArea > div > div.product_rating_area.review-write-delete > div > div.star_area > p.num > strong').text
  except:
      print('별점은없어')  # 리뷰가 없는경우 tstar를 가져오지 못함.(리뷰가 없음)
      pname = driver.find_element(By.XPATH,'//*[@id="Contents"]/div[2]/div[2]/div/p[2]').text


In [ ]:
# 브랜드명과 가격정보
def rv_brand():
  try:
    global brand
    brand = driver.find_element(By.CLASS_NAME, 'prd_brand').text
    global price
    price = driver.find_element(By.CLASS_NAME, 'price').text
  except:
    brand = '-'
    price = '-'

In [ ]:
## 리뷰의 키워드를 반환해주는 함수

def find_kw(i):
    try:  # css가 특징이 정확함
        x = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.poll_sample ').text
    except:
        x = 'no keyword' # 키워드 선택을 안하는 경우
    finally:
        kw_l.append(x)

In [ ]:
## review text
def find_text(i):   # //*[@id="gdasList"]/li[1]/div[2]/div[2]/p   헤드 없고 키워드 없고 내용만  
        # 키워드 없는 헤드 //*[@id="gdasList"]/li[6]/div[2]/p    ## //*[@id="gdasList"]/li[6]/div[2]/div[2]   # 내용인데 헤드가 있으면
    try:
        text = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.txt_inner').text
        # txt inner
        # rv = x.split(sep='\n')
        # if len(rv) == 1: # 리뷰가 한줄로 작성된 경우
        #     text = rv[0]
        # else:
        #     text = rv[0]+ rv[-1] # 리뷰 앞문장과 마지막 문장 가져오기
        # print(text)
    except:
        try:  # 리뷰 아우터만 존재하는 경우
            text = driver.find_element(By.CSS_SELLECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > p').text
        except:
            text = 'no text'  # 해당 리뷰 구조가 없으면 리뷰가 없는걸로
    finally:
        rv_l.append(text)

In [ ]:
## 한페이지 안에 가져올 리뷰정보를 반환하는 함수
def list_up(n=10):  # 파라미터는 한 페이지 안에 있는 리뷰수
    for i in range(1, n+1):
        time.sleep(0.1)
        x = driver.find_element(By.XPATH, f'//*[@id="gdasList"]/li[{i}]/div[2]/div[1]/span[1]/span').text
        x = list(x)
        star = int(x[-2])
        star_l.append(star)
        find_kw(i)   # keywords function
        find_text(i) # 리뷰 함수
        brand_l.append(brand)
        cate_l.append(cate)
        pname_l.append(pname)
        price_l.append(price)
        tstar_l.append(tstar)
        # print('ok')

In [ ]:
## 한 상품당 최대 30개까지 가져오기
def one_product(pn, pc):
     # 가장 첫페이지 리뷰가져오기
    list_up()
    print('1_page')
    for l in range(2, 4): # 리뷰 페이지 수 만큼 
        if l == (pn): # 현재가 마지막 페이지
            driver.find_element(By.LINK_TEXT, f'{l}').click()
            moment
            if pc != 0:
                list_up(pc)
                print(l,'_page')
            else:
                list_up() # 0으로 끝나면 10이니
                print(l,'_page')
            # 끝자리수만큼 포문 pc list_up(pc)
            # print(l)
        elif '2' <= str(l)[-1] <= '9' and l != pn: # 0,1 마지막 페이지가 아닌경우 다음버튼 누른후 10개씩 가져옴
            driver.find_element(By.LINK_TEXT, f'{l}').click()
            moment
            list_up()
            print(l,'_page')

In [ ]:
## 한 상품에서 가져올 정보를 반환하는 함수(최대한 많이)
def one_product2(pn, pc):
     # 가장 첫페이지 리뷰가져오기
    list_up()
    print('1_page')
    for l in range(2, 4): # 리뷰 페이지 수 만큼 
        if str(l)[-1] == '0':
            try:  # 10, 20 등 뒤에 다음 버튼이 오는 경우
                driver.find_element(By.LINK_TEXT, f'{l}').click() # 페이지 넘검
                moment
                list_up() # 리뷰내용 가져오는 함수(default = 10)
                print(l,'_page')
                driver.find_element(By.PARTIAL_LINK_TEXT, f'다음').click() # 다음 페이지로
                moment
                a = driver.find_elements(By.CLASS_NAME, 'review_cont') # 해당 페이지의 리뷰수 카운트
                if len(a) == 10: # 리뷰수 10개
                    list_up()
                else: 
                    list_up(pc) # pc랑 a가 같은 수임
                print(l,'_page')
            except: # 클릭후 다음 버튼이 없는 경우
                driver.find_element(By.LINK_TEXT, f'{l}').click() # 마지막 페이지
                moment
                a = driver.find_elements(By.CLASS_NAME, 'review_cont') # 해당 페이지의 리뷰수 카운트
                list_up(len(a))  # pc가 0일 수도 있어서
                print(l, '_page')
                
        elif str(l)[-1] == '0' and l == pn: # 다음페이지가 마지막인 경우
            driver.find_element(By.LINK_TEXT, f'{l}').click()
            moment
            list_up()
            print(l,'_page')
            driver.find_element(By.PARTIAL_LINK_TEXT, f'다음').click()
            moment
            if pc != 0: # pc가 0이면 for문 안돌아가서 
                list_up(pc)
                print(l,'_page^')
            else:
                list_up()
                print(l,'_page')
        elif str(l)[-1] == '1': 
            print(l) # 11,21 등의 페이지는 버튼클릭이 다음 버튼이라 넣을 내용이 없음.
        elif l == (pn): # 현재가 마지막 페이지
            driver.find_element(By.LINK_TEXT, f'{l}').click()
            moment
            if pc != 0:
                list_up(pc)
                print(l,'_page')
            else:
                list_up() # 0으로 끝나면 10이니
                print(l,'_page')
            # 끝자리수만큼 포문 pc list_up(pc)
            # print(l)
        elif '2' <= str(l)[-1] <= '9' and l != pn: # 0,1 마지막 페이지가 아닌경우 다음버튼 누른후 10개씩 가져옴
            driver.find_element(By.LINK_TEXT, f'{l}').click()
            moment
            list_up()
            print(l,'_page')

In [ ]:
## 상품의 리뷰를 가져오는 함수
### 위 함수들의 진행하는데 있어 생각하지 못한 오류 들이 발생할 수 있어서 예외처리하고자 함.
def try_pro():
  try:
      rv_num2()
      rv_info2()
      rv_brand()  # 새로 추가됨(브랜드, 가격)
      if pn == 'zero' and pc == 'zero':
            # 상품리뷰 없음. 
          non_l.append(pname) # 상품명 리스트에는 안올라감
      elif pn == 'one':
          # 1~10사이
        list_up(pc) # 10개라도 여기선 pc가 q임 ㅎ
        print(pname, '_수집')
      else:
        one_product(pn, pc)     # one_product2(pn, pc)
        print(pname, '_수집')
  except:
        # pname = driver.find_element(By.XPATH,'//*[@id="Contents"]/div[2]/div[2]/div/p[2]').text
      fail_l.append(pname)
      print("??? 문제 발생 ???", pname)

# action

In [ ]:
# page moving
driver.get(url=a)
driver.implicitly_wait(5)
# 기본페이지 이동
# 스킨케어_토너 페이지로 이동
driver.find_element(By.XPATH, f'//*[@id="Contents"]/div/div[1]/ul/li[1]/a').click()
driver.implicitly_wait(5)
# 리뷰 48개로 선택
# driver.find_element(By.XPATH, '//*[@id="Contents"]/div[5]/div[2]/ul/li[3]/a').click()
# driver.implicitly_wait(5)

# 리뷰 컨테이너 요소를 만나기 위한 작업(explicitly wait지정)
driver.find_element(By.XPATH,f'//*[@id="Contents"]/ul[{2}]/li[{1}]/div/a/img').click()
driver.implicitly_wait(5)
driver.find_element(By.XPATH,f'//*[@id="reviewInfo"]/a').click()
## 리뷰컨테이너를 만날때까지 최대 6초 기다림
moment = WebDriverWait(driver, 6).until(EC.presence_of_element_located((By.CLASS_NAME, 'review_cont'))) # 해당요소를 만나야만 가능

# 첫페이지 화면으로
driver.back()

In [ ]:
# 리스트 정의
# stored information
cate_l = []
pname_l = []
brand_l = []
price_l = []
tstar_l = []
star_l = []
kw_l = []
rv_l = []

non_l = []
fail_l = []

# 링크 저장
link_l = [ab, ac, ba, bb, bc]
# 카테고리 저장
catelist = ['에센스', '미스트','립','베이스','아이']

In [ ]:
print('시작 시간: ',datetime.now(timezone('Asia/Seoul')))

시작 시간:  2023-01-28 18:58:52.849712+09:00


In [ ]:
# crawler 

# 처음 수집되는 카테고리
cate = '토너'

# 6개의 카테고리당 20개의 상품정보 수집
for idx, x in enumerate(link_l):
    try:
        for i in range(1, 6):
            for j in range(1,5):
                # 상품 선택
                driver.find_element(By.XPATH,f'//*[@id="Contents"]/ul[{i+1}]/li[{j}]/div/a/img').click()
                driver.implicitly_wait(5)
                # 리뷰칸 선택
                driver.find_element(By.XPATH,f'//*[@id="reviewInfo"]/a').click()
                moment
                try_pro()
                driver.back()
                driver.implicitly_wait(5)
    except:
        print('어디서 오류가 났으려나? 일단 pass^^')
    try:
        driver.get(url=x)
        moment
        # driver.find_element(By.XPATH, '//*[@id="Contents"]/div[5]/div[2]/ul/li[3]/a').click() # 48 선택
        # driver.implicitly_wait(5)
        # 카테고리 지정
        cate = catelist[idx]

    except:
        continue

1_page
2 _page
3 _page
[대용량+미니어처] 넘버즈인 3번 결광가득 에센스 토너 300ml 어워즈 한정기획 (+3번 결광케어 키트 5종 증정) _수집
1_page
2 _page
3 _page
[2022어워즈]아누아 어성초 77 토너 어워즈 한정 기획 (500ml+250ml) _수집
1_page
2 _page
3 _page
[1월한정 초특가] 피지오겔 DMT 로션 200ml 기획(+토너 50ml 증정) _수집
1_page
2 _page
3 _page
[증량] 더랩바이블랑두 저분자 히알루론산 토너(본품500ml+100ml 추가 증정) _수집
1_page
2 _page
3 _page
[대용량] 라운드랩 1025 독도 토너 (본품500ml+100ml 추가 증정) _수집
1_page
2 _page
3 _page
에스트라 아토베리어365 로션 150ml _수집
1_page
2 _page
??? 문제 발생 ??? [스완PICK] 라운드랩 포 맨 자작나무 토너/로션 2종세트(+미니어처 2종 증정)
1_page
2 _page
3 _page
[선물 기획] 바이오힐보 프로바이오덤 리페어 스킨&에멀전 2종 기획 (본품2종+크림10ml+아이크림10ml 증정) _수집
1_page
2 _page
3 _page
우르오스 올인원 200ml 2종 택 1 _수집
1_page
2 _page
3 _page
[단독기획] 토리든 다이브인 저분자 히알루론산 토너 300ml 기획(+100ml 추가 증정) _수집
1_page
2 _page
3 _page
아비브 어성초 카밍 토너 스킨부스터 더블 기획 (200ml+200ml) _수집
1_page
2 _page
3 _page
아누아 어성초 77토너 + 어성초 70 로션 듀오 기획세트 _수집
1_page
2 _page
3 _page
[어성초] 넘버즈인 1번 진정 맑게담은 청초토너 300ml 기획 (+1번 패드 10매 증정) _수집
(NEW)라네즈 크림스킨 170ml+120ml 리필 기획 _수집
1_page
??? 문제 발생 ??? 우르오스 스킨밀

In [ ]:
# driver.close()

df = pd.DataFrame({'cate': cate_l,'brand': brand_l, 'pname':pname_l,'price':price_l, 'tstar':tstar_l,'star':star_l,'keyword':kw_l, 'rv':rv_l})
df.to_csv('/content/drive/MyDrive/data_DB.csv', sep = ',', encoding = 'utf-8-sig', index=False)  # 카테고리 포함 버전

In [ ]:
print('끝난 시간: ',datetime.now(timezone('Asia/Seoul')))

끝난 시간:  2023-01-28 19:34:22.596662+09:00


In [ ]:
### 아이가 안뽑힘 
cate = '아이'

driver.get(url = bc)
try:
    for i in range(1, 6):
        for j in range(1,5):
                # 상품 선택
            driver.find_element(By.XPATH,f'//*[@id="Contents"]/ul[{i+1}]/li[{j}]/div/a/img').click()
            driver.implicitly_wait(5)
              # 리뷰칸 선택
            driver.find_element(By.XPATH,f'//*[@id="reviewInfo"]/a').click()
            moment
            try_pro()
            driver.back()
            driver.implicitly_wait(5)
except:
    print('어디서 오류가 났으려나? 일단 pass^^')

??? 문제 발생 ??? 클리오 샤프, 쏘 심플 워터프루프 펜슬라이너
1_page
2 _page
3 _page
클리오 킬래쉬 수퍼프루프마스카라 1+1 기획 _수집
1_page
2 _page
3 _page
[NEW] 컬러그램 올인원 애교살 메이커 5종 _수집
1_page
2 _page
3 _page
[2022어워즈] 데이지크 섀도우 팔레트 (1~16호 단품/ 브러쉬,네일,어워즈 기획) _수집
1_page
2 _page
3 _page
[2022어워즈] 코스노리 아이래쉬 세럼 속눈썹영양제 기획세트 (+손톱영양제 7ml 증정) _수집
1_page
2 _page
3 _page
[민니 Pick] 3CE 멀티 아이 컬러 팔레트 _수집
1_page
2 _page
3 _page
에스쁘아 더브로우 밸런스 펜슬 _수집
1_page
2 _page
3 _page
웨이크메이크 철벽 펜 아이라이너 3color (1+1/단품) _수집
1_page
2 _page
??? 문제 발생 ??? 롬앤 한올 샤프 브로우
1_page
2 _page
3 _page
클리오 킬브로우 오토하드브로우펜슬 _수집
[NEW/단독] 에뛰드 컬 픽스 마스카라 트위저 증정기획 3colors _수집
1_page
2 _page
3 _page
컬러그램 음영 창조 라이너 _수집
1_page
2 _page
3 _page
클리오 쉐이드앤섀도우 팔레트 _수집
1_page
2 _page
3 _page
페리페라 스피디 스키니 브로우 _수집
1_page
2 _page
3 _page
[문빈PICK] 지베르니 밀착 센서티브 마스카라 1+1 기획 (본품+본품) _수집
1_page
2 _page
??? 문제 발생 ??? 에스쁘아 리얼아이팔레트
1_page
2 _page
3 _page
[더블기획/단품]클리오 수퍼프루프 펜라이너 _수집
1_page
2 _page
3 _page
[1+1기획] 퓌 제이컬링 마스카라 1+1 기획 _수집
1_page
2 _page
3 _page
홀리카홀리카 래쉬코렉팅마스카라 1+1 기획 (9ml+9ml) _수집
1_pag

In [ ]:
df = pd.DataFrame({'cate': cate_l,'brand': brand_l, 'pname':pname_l,'price':price_l, 'tstar':tstar_l,'star':star_l,'keyword':kw_l, 'rv':rv_l})
df.to_csv('/content/drive/MyDrive/data_DBeye.csv', sep = ',', encoding = 'utf-8-sig', index=False)  # 카테고리 포함 버전
driver.close()